# Tombola maker

Since I couldn't play tombola IRL with my friends, I coded this Tombola cartella maker + number extractor.

Why? Because.

In [1]:
import numpy as np

### Game Functions

In [30]:
def is_valid(cartella) -> bool:
    valid = True
    for row in cartella:
        valid &= np.sum([1 for i in row if i != 0]) == 5
    return valid

In [31]:
def clean_row(row, i, cartella):
    diff = np.sum([1 for i in row if i != 0]) - 5
    while diff > 0:
        for j, v in enumerate(row):
            idx = i + 1 if i+1<3 else i-1
            if cartella[idx, j] == 0:
                cartella[idx, j] = v
                cartella[i, j] = 0
                diff -= 1
            if diff == 0:
                break

In [29]:
def get_cartella() -> np.ndarray:
    rng = np.random.default_rng()
    cartella = np.zeros(shape=(3,9), dtype='int')  # a cartella is a 3x9 grid
    n_for_file = np.zeros(shape=9, dtype=int)
    # a cartella contains 15 values
    while np.sum(n_for_file) != 15:  # is there a better way? Probably. Do I want to spend more time on it? No.
        n_for_file = np.random.randint(low=1, high=3+1, size=9)
    # this is a "cheat" way to get the numbers sorted already
    for i, n in enumerate(n_for_file):
        rand = np.sort(rng.choice(np.arange((i*10)+1, (i+1)*10), size=3, replace=False)[:n])
        cartella[:n, i] = rand
    # only 5 numbers per row
    while not is_valid(cartella):
        for i, row in enumerate(cartella):
            clean_row(row, i, cartella)
    return cartella

In [34]:
def set_cartelle() -> None:
    global players

    for p in players.keys():
        for n in range(players[p].get('n_cartelle', 1)):
            players[p]['cartelle'].append(get_cartella())

In [35]:
def print_cartelle(player: str) -> None:
    global players

    cartelle = players[player]['cartelle']
    print(f'{player}\'s cartelle:')
    for cartella in cartelle:
        s = ''
        for r in cartella:
            for v in r:
                s += f'| {str(v).zfill(2) if v != 0 else "  "} '
            s += '|\n'
        print(s)

In [36]:
def add_player(name: str,
               n_cartelle: int) -> None:
    global players

    players[name] = {}
    players[name]['n_cartelle'] = n_cartelle
    players[name]['cartelle'] = []

In [37]:
def extract() -> int:
    global extracted

    assert len(extracted) <= 90, 'How has no one won yet? Impossible!'

    n = 0
    if len(extracted) == 0:
        n = np.random.randint(1, 90+1)
        extracted.add(n)
    else:
        invalid = True
        while invalid:
            n = np.random.randint(1, 90+1)
            invalid &= n in extracted
        extracted.add(n)
    return n

### Game setup

Global variables

In [38]:
players = dict()
extracted = set()

Modify these to setup the game

In [39]:
pnames = ['test']
pcartelle = [1]
 
assert len(pnames) == len(pcartelle), 'Make sure each player has a specific number of cartelle to play with!'

Run setup

In [40]:
for pname, n in zip(pnames, pcartelle):
    add_player(name=pname,
               n_cartelle=n)
 
set_cartelle()

Print cartelle

In [41]:
for p in players.keys():
    print('---')
    print_cartelle(p)

---
test's cartelle:
| 03 |    | 22 |    | 45 | 53 |    |    | 81 |
| 05 |    |    |    | 47 | 56 | 61 | 72 |    |
| 09 | 16 | 27 | 32 | 48 |    |    |    |    |



Run this cell to extract the numbers

In [45]:
print(f'Extracted number is: {extract()} (remaining: {90 - len(extracted)})')

Extracted number is: 88 (remaining: 88)


Run this cell to see all previously extracted numbers

In [46]:
sorted = list(extracted)
sorted.sort()
print(f'All previously extracted numbers: {sorted}')

All previously extracted numbers: [27, 88]
